# Creating First Graph and Running in a Session

In [1]:
import tensorflow as tf

x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')
f = x * x * y + y + 2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [3]:
sess.close()

In [4]:
# This is clumsy. The following is an easier way, using context manager:

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)
    

42


In [6]:
# Instead of manually initializing each variable, can use global_variables_initalizer() function:

init = tf.global_variables_initializer()  # prepare an init node

with tf.Session() as sess:
    init.run()  # actually initialize all the variables
    print(f.eval())

42


In [7]:
# Using an interactive session instead

sess = tf.InteractiveSession()
init.run()
print(f.eval())
sess.close()

42


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# Managing Graphs

In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [9]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)


In [10]:
x2.graph is graph

True

In [11]:
x2.graph is tf.get_default_graph()

False

# Lifecycle of a Node Value

In [12]:
# Consider the following code:

w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

10
15


In [13]:
# The above evaluates x twice. If we want to do it efficiently, must evaluate y and z in
# just one graph run, as below:

with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


# Linear Regression

In [10]:
# This will implement Linear Regression on the California housing dataset from way back

import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:749)>

# Gradient Descent

In [9]:
# Scale the data first using sklearn!

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

NameError: name 'housing' is not defined

In [15]:


print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)



NameError: name 'scaled_housing_data_plus_bias' is not defined

In [11]:
n_epoches = 100
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = 'y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    

NameError: name 'scaled_housing_data_plus_bias' is not defined

In [12]:
# Instead, we can use tensorflow's autodiff feature. Replace the previous line starting with gradients = 
# with:

gradients = tf.gradients(mse, [theta])[0]

NameError: name 'mse' is not defined

## Feeding Data to the Training Algorithm

Let's try to modify the previous code to implement Mini-batch Gradient Descent.

In [13]:
# This will require placeholder nodes

A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict = {A: [[4, 5, 6], [7, 8, 9]]})
    
# The feed_dict method allows us to specify the value of A
print(B_val_1)

[[6. 7. 8.]]


In [14]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [ ]:
# Change the def of X and y in construction to make them placeholders

X = tf.placeholder(tf.float32, shape = (None, n + 1), name = "X")
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

# Define the batch size and compute total number of batches

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

# Fetch the mini-batches one by one, then provide the value of X and y via feed_dict param when evaluating a
# node that depends on either of them

def fetch_batch(epoch, batch_index, batch_size):
    # Load the data from disk
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
            
    best_theta = theta.eval()

## Saving and Restoring Models

It'd be useful to save our training results. TF makes this easy with `Saver()` nodes:

In [ ]:
# stuff
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = 'theta')
# more stuff
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:  # Checkpoint every 100 epochs
            save_path = saver.save(sess, '/tmp/my_model.ckpt')
            
        sess.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/my_model_final.ckpt')


Restoration of a previous training session is just as easy. Create a `Saver` at the end of the construction phase just like before, and at beginning of execution, instead of initializing the variables using `init` node, call `restore()` method of the `Saver` object:

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_final.ckpt')
    
    # etc
    
# Default Saver saves and restores all variables, if need more control, can specify which variables to save or restore
# and what names to use

saver = tf.train.Saver({'weights' : theta})

# Default Saver also saves structure of the graph in a second .meta file. Can load this structure and add it to
# the default graph, returns a Saver instance can be used to restore the variable values

saver = tf.train.import_meta_graph('/tmp/my_model_final.ckpt.meta')

with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_model_final.ckpt')
    
# This gives us the full restoration of saved model, including graph structure and variable values

## Visualizing the Graph and Training Curves Using TensorBoard

TensorBoard allows for visualizations of the training and stats. All we need to do to use it is change our program
so it provides some basic stats in a log directory, one for each time we train.

In [ ]:
# Put this in the beginning of the program
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)

# Next, add the following at the end of the contruction phase

mse_summary = tf.summary.scalar('MSE', mse)  # Evaluate MSE and write it in TensorBoard log file called a summary
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
# ^ create a FileWriter that writes summaries to logfiles in the log dir


Next we need to update the execution phase to evaluate the `mse_summary` node regularly during training. This outputs a summary that can be written to the events file using the `file_writer`. 

In [ ]:
# blah
for batch_index in range(n_batches):
    X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
    if batch_index % 10 == 0:
        summary_str = mse_summary.eval(feed_dict = {X: X_batch, y: y_batch})
        step = epoch * n_batches + batch_index
        file_writer.add_summary(summary_str, step)
    sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
# blah

# Finally, make sure to close the FileWriter at the end of the program
file_writer.close()


## Name Scopes

The graph can get really messy, so we can create _name scopes_ to group related nodes. Let's modify the previous
code to define the `error` and `mse` ops within a name scope called "loss".

In [ ]:
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = 'mse')
    
# The name of each op defined within the scope is now prefixed with 'loss/'


## Modularity

Say we want to create a graph that adds the output of two _rectified linear units_ (ReLU). The following code does the job, but it's repetitive:

In [ ]:
n_features = 3
X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')

w1 = tf.Variable(tf.random_normal((n_features, 1)), name = 'weights1')
w2 = tf.Variable(tf.random_normal((n_features, 1)), name = 'weights2')
b1 = tf.Variable(0.0, name = 'bias1')
b2 = tf.Variable(0.0, name = 'bias2')

z1 = tf.add(tf.matmul(X, w1), b1, name = 'z1')
z2 = tf.add(tf.matmul(X, w2), b2, name = 'z2')

relu1 = tf.maximum(z1, 0., name = 'relu1')
relu2 = tf.maximum(z2, 0., name = 'relu2')

output = tf.add(relu1, relu2, name = 'output')

# This is bad code. TensorFlow lets us create a function to build a ReLU. The following creates 5 ReLUs and outputs
# their sum:

def relu(X):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_uniform(w_shape), name = 'weights')
        b = tf.Variable(0.0, name = bias)
        z = tf.add(tf.matmul(X, w), b, name = 'z')
        return tf.maximum(z, 0., name = 'relu')

n_features = 3
X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name = 'output')

## Sharing Variables

If we want to share a variable amongst components of the graph, a simple solution is to create it first, then pass it as a parameter to the functions that need it. Say we wanted to control the ReLU threshold using a shared `threshold` variable for all of them. We could create the variable first, then pass it to the `relu()` function:

In [15]:
def relu(X, threshold):
    with tf.name_scope('relu'):
        #blah
        return tf.maximum(z, threshold, name = 'max')
    
threshold = tf.Variable(0.0, name = 'threshold')
X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')
relus = [relu(X, threshold) for _ in range(5)]
output = tf.add_n(relus, name = 'ouput')

# We can set the shared variable as an attr of the relu() function upon the first call:

def relu(X):
    with tf.name_scope('relu'):
        if not hasattr(relu, 'threshold'):
            relu.threshold = tf.Variable(0.0, name = 'threshold')
        # blah
        return tf.maximum(z, relu.threshold, name = 'max')
    


NameError: name 'n_features' is not defined

In [ ]:
# There's another option. Get get_variable() to create the shared variable if it doesn't exist, and reuse else. This is
# controlled by an attr of the current variable_scope()

# This will create a variable named 'relu/threshold'

with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshold', shape = ()
                                initializer = tf.constant_initializer(0.0))
    
# This raises an exception if the variable has been created before. If we want to reuse, have to specify scope's reuse
# variable to True

with tf.variable_scope('relu', reuse = True):
    threshold = tf.get_variable('threshold')
    
# This will fetch existing '/relu/threshold' variable, or raise exception if doesn't exist, or wasn't created using
# get_variable(). Can also set reuse attr to True inside the block by calling scope's reuse_variables() method:

with tf.variable_scope('relu') as scope:
    scope.reuse_variables()
    threshold = tf.get_variable('threshold')
    

In [ ]:
# Now we have all the pieces we need to make the relu() function access the threshold variable without having to 
# pass it as a parameter:

def relu(X):
    with tf.variable_scope('relu', reuse = True):
        threshold = tf.get_variable('threshold')   # reuse existing variable
        # blah
        return tf.maximum(z, threshold, name = 'max')
    
X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')
with tf.variable_scope('relu'):   # Create the variable
    threshold = tf.get_variable('threshold', shape = ()
                                initializer = tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name = 'output')


In [ ]:
# Let's add threshold to the relu function, and not define it outside. So, we'll create the threshold variable within
# the relu() function on the first call, then reuse it afterwards. Rest of the code calls relu() five times, with
# reuse = False the first time, and reuse = True the other times

def relu(X):
    threshold = tf.get_variable('threshold', shape = ()
                                initializer = tf.constant_initializer(0.0))
    # blah
    return tf.maximum(z, threshold, name = 'max')

X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')
relus = []
for relu_index in range(5):
    with tf.variable_scope('relu', reuse = (relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name = 'output')

## Exercises

### `#` 10

In [16]:
import tensorflow as tf

x = tf.Variable(tf.random_uniform(shape = (), minval = 0.0, maxval = 1.0))
x_new_val = tf.placeholder(shape = (), dtype = tf.float32)
x_assign = tf.assign(x, x_new_val)

with tf.Session():
    x.initializer.run()  # random number is sampled now
    print(x.eval())  
    x_assign.eval(feed_dict = {x_new_val: 5.0})
    print(x.eval())

0.7944101
5.0


### `#` 11

In [30]:
import numpy as np
from sklearn.datasets import make_moons

# Load the data
m = 1000

n_epochs = 1000
learning_rate = 0.01

X_moons, y_moons = make_moons(m, noise = 0.1, random_state = 42)
X_moons_plus_bias = np.c_[np.ones((m, 1)), X_moons]
y_moons_column_vector = y_moons.reshape(-1, 1)

# Split into training and test set
test_ratio = 0.2
test_size = int(m * test_ratio)
X_train = X_moons_plus_bias[:-test_size]
X_test = X_moons_plus_bias[-test_size:]
y_train = y_moons_column_vector[:-test_size]
y_test = y_moons_column_vector[-test_size:]


# Now create a function to make random samples of the training
def random_batch(X_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[rnd_indices]
    y_batch = y_train[rnd_indices]
    return X_batch, y_batch



In [33]:
X_batch, y_batch = random_batch(X_train, y_train, 5)
X_batch

array([[ 1.        , -1.06204208,  0.40739827],
       [ 1.        ,  0.53077648,  0.97543651],
       [ 1.        ,  0.0971698 ,  0.49928906],
       [ 1.        , -0.27089595,  0.86478076],
       [ 1.        , -0.88517979, -0.01783117]])

In [34]:
y_batch

array([[0],
       [0],
       [1],
       [0],
       [0]])

In [36]:
n_inputs = 2

# set up the nodes
X = tf.placeholder(tf.float32, shape = (None, n_inputs + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
theta = tf.Variable(tf.random_uniform([n_inputs + 1, 1], -1.0, 1.0, seed = 42), name = 'theta')
logits = tf.matmul(X, theta, name = 'logits')
y_proba = tf.sigmoid(logits)



In [37]:
# trying to minimize log loss
loss = tf.losses.log_loss(y, y_proba)

In [ ]:
# tf lets us easily perform gradient descent
learning_rate = 0.01
optimizier = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)

In [40]:
# get all the variables set up
init = tf.global_variables_initializer()

In [41]:
# now run, pretty much as before. nothing special added yet

n_epochs = 1000
batch_size = 50
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = random_batch(X_train, y_train, batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
        loss_val = loss.eval({X: X_test, y: y_test})
        if epoch % 100 == 0:
            print("Epoch:", epoch, "\tLoss:", loss_val)
            
        y_proba_val = y_proba.eval(feed_dict = {X: X_test, y: y_test})
            

Epoch: 0 	Loss: 0.87446475
Epoch: 100 	Loss: 0.35141128
Epoch: 200 	Loss: 0.31101328
Epoch: 300 	Loss: 0.29459465
Epoch: 400 	Loss: 0.2858931
Epoch: 500 	Loss: 0.2814843
Epoch: 600 	Loss: 0.2784729
Epoch: 700 	Loss: 0.27660805
Epoch: 800 	Loss: 0.2755033
Epoch: 900 	Loss: 0.27441597


In [42]:
# Let's add the squares and cubes of our features to improve the model
X_train_enhanced = np.c_[X_train,
                         np.square(X_train[:, 1]),
                         np.square(X_train[:, 2]),
                         X_train[:, 1] ** 3,
                         X_train[:, 2] ** 3]

X_test_enhanced = np.c_[X_test,
                        np.square(X_test[:, 1]),
                        np.square(X_test[:, 2]),
                        X_test[:, 1] ** 3,
                        X_test[:, 2] ** 3]



In [44]:
# Now define the logistic_regression() function.

def logistic_regression(X, y, initializer = None, seed = 42, learning_rate = 0.01):
    n_inputs_including_bias = int(X.get_shape()[1])
    with tf.name_scope('logistic_regression'):
        with tf.name_scope('model'):
            if initializer is None:
                initializer = tf.random_uniform([n_inputs_including_bias, 1], -1.0, 1.0, seed = seed)
                
            theta = tf.Variable(initializer, name = 'theta')
            logits = tf.matmul(X, theta, name = 'logits')
            y_proba = tf.sigmoid(logits)
        with tf.name_scope('train'):
            loss = tf.losses.log_loss(y, y_proba, scope = 'loss')
            optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
            training_op = optimizer.minimize(loss)
            loss_summary = tf.summary.scalar('log_loss', loss)
        with tf.name_scope('init'):
            init = tf.global_variables_initializer()
        with tf.name_scope('save'):
            saver = tf.train.Saver()
        
    return y_proba, loss, training_op, loss_summary, init, saver


            
            

In [45]:
# Create a function to get the name of the log directory to save the summaries for TensorBoard
from datetime import datetime

def log_dir(prefix = ""):
    now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
    root_logdir = 'tf_logs'
    if prefix:
        prefix += '-'
    name = prefix + 'run-' + now
    return '{}/{}/'.format(root_logdir, name)

In [46]:
# Create the graph using our function. Also create FileWriter to save summaries to log directory or TensorBoard

n_inputs = 2 + 4
logdir = log_dir('logreg')

X = tf.placeholder(tf.float32, shape = (None, n_inputs + 1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

y_proba, loss, training_op, loss_summary, init, saver = logistic_regression(X, y)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


In [49]:
# Train the model!
import os

n_epochs = 10001
batch_size = 50
n_batches = int(np.ceil(m / batch_size))

checkpoint_path = '/tmp/my_logreg_model.ckpt'
checkpoint_epoch_path = checkpoint_path + '.epoch'
final_model_path = './my_logreg_model'

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if checkpoint exists, restore the model and load epoch number
        with open(checkpoint_epoch_path, 'rb') as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
        
    for epoch in range(start_epoch, n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = random_batch(X_train_enhanced, y_train, batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
        loss_val, summary_str = sess.run([loss, loss_summary], feed_dict = {X: X_test_enhanced, y: y_test})
        file_writer.add_summary(summary_str, epoch)
        if epoch % 500 == 0:
            print("Epoch:", epoch, "\tLoss:", loss_val)
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, 'wb') as f:
                f.write(b'%d' % (epoch + 1))
                
    saver.save(sess, final_model_path)
    y_proba_val = y_proba.eval(feed_dict = {X: X_test_enhanced, y: y_test})
    os.remove(checkpoint_epoch_path)
    
        

Epoch: 0 	Loss: 0.8125505
Epoch: 500 	Loss: 0.17470044
Epoch: 1000 	Loss: 0.12627645
Epoch: 1500 	Loss: 0.1019548
Epoch: 2000 	Loss: 0.08697619
Epoch: 2500 	Loss: 0.076763995
Epoch: 3000 	Loss: 0.0693285
Epoch: 3500 	Loss: 0.06366068
Epoch: 4000 	Loss: 0.05915987
Epoch: 4500 	Loss: 0.055509605
Epoch: 5000 	Loss: 0.05249735
Epoch: 5500 	Loss: 0.049933728
Epoch: 6000 	Loss: 0.04775853
Epoch: 6500 	Loss: 0.045840207
Epoch: 7000 	Loss: 0.044198904
Epoch: 7500 	Loss: 0.042698365
Epoch: 8000 	Loss: 0.041369144
Epoch: 8500 	Loss: 0.04023262
Epoch: 9000 	Loss: 0.039146718
Epoch: 9500 	Loss: 0.03816019
Epoch: 10000 	Loss: 0.03726425


In [51]:
# A simple implementation of randomized search of the hyperparameters. Checkpoints were removed for simplicity.

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
    
from scipy.stats import reciprocal

from sklearn.metrics import precision_score, recall_score

n_search_iterations = 10

for search_iteration in range(n_search_iterations):
    batch_size = np.random.randint(1, 100)
    learning_rate = reciprocal(0.0001, .1).rvs(random_state = search_iteration)
    
    n_inputs = 2 + 4
    logdir = log_dir('logreg')
    
    print("Iteration", search_iteration)
    print("  logdir:", logdir)
    print("  batch size:", batch_size)
    print("  learning_rate:", learning_rate)
    print("  training: ", end="")
    
    reset_graph()
    
    X = tf.placeholder(tf.float32, shape = (None, n_inputs + 1), name = 'X')
    y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')
    
    y_proba, loss, training_op, loss_summary, init, saver = logistic_regression(X, y, learning_rate = learning_rate)
    
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    n_epochs = 10001
    n_batches = int(np.ceil(m /  batch_size))
    
    final_model_path = './my_logreg_model_%d' % search_iteration
    
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(n_epochs):
            for batch_index in range(n_batches):
                X_batch, y_batch = random_batch(X_train_enhanced, y_train, batch_size = batch_size)
                sess.run(training_op, feed_dict = {X: X_train_enhanced, y: y_train})
                
            loss_val, summary_str = sess.run([loss, loss_summary], feed_dict = {X: X_test_enhanced, y: y_test})
            file_writer.add_summary(summary_str, epoch)
            if epoch % 500 == 0:
                print(".", end = "")
                
        saver.save(sess, final_model_path)
        
        print()
        y_proba_val = y_proba.eval(feed_dict = {X: X_test_enhanced, y: y_test})
        y_pred = (y_proba_val >= 0.5)
        
        print("  precision:", precision_score(y_test, y_pred))
        print("  recall:", recall_score(y_test, y_pred))
            
        
    

Iteration 0
  logdir: tf_logs/logreg-run-20180809005652/
  batch size: 37
  learning_rate: 0.004430375245218265
  training: .....................
  precision: 0.9797979797979798
  recall: 0.9797979797979798
Iteration 1
  logdir: tf_logs/logreg-run-20180809005840/
  batch size: 58
  learning_rate: 0.0017826497151386947
  training: .....................
  precision: 0.9696969696969697
  recall: 0.9696969696969697
Iteration 2
  logdir: tf_logs/logreg-run-20180809005950/
  batch size: 61
  learning_rate: 0.00203228544324115
  training: .....................
  precision: 0.9696969696969697
  recall: 0.9696969696969697
Iteration 3
  logdir: tf_logs/logreg-run-20180809010056/
  batch size: 92
  learning_rate: 0.004491523825137997
  training: .....................
  precision: 0.9797979797979798
  recall: 0.9797979797979798
Iteration 4
  logdir: tf_logs/logreg-run-20180809010142/
  batch size: 74
  learning_rate: 0.07963234721775589
  training: .....................
  precision: 0.980198019801